In [1]:
import math
import random
import matplotlib.pyplot as plt
import string
import numpy as np

# Wadsworth

## Function Definitions

In [2]:
def encode_wadsworth(pt_a, ct_a, pt, delta_func=lambda d, pt, l : d):
  pt_index = 0
  ct_index = 0
  ct = []

  for l in pt:
    pt_l_index = pt_a.index(l)
    pt_delta = pt_l_index - pt_index
    if pt_l_index < pt_index: pt_delta += len(pt_a)
    if pt_l_index == pt_index: pt_delta += len(pt_a)
    ct_delta = delta_func(pt_delta, pt, l)

    pt_index = (pt_index + pt_delta) % len(pt_a)
    ct_index = (ct_index + ct_delta) % len(ct_a)
    ct.append(ct_a[ct_index])

  return ct

In [3]:
def simulate_annealing(init_temp, k_max, it_max, bestRetention, f_init, f_nudge, f_fitness):
  best_state = f_init()
  best_state_fitness = f_fitness(best_state)
  best_states = [ ( best_state_fitness, best_state ) ]

  for k in range(k_max):
    T = init_temp * (1 - k / k_max)

    for it in range(it_max):
      candidate = f_nudge(best_state)
      candidate_fitness = f_fitness(candidate)
      df = candidate_fitness - best_state_fitness

      if (df > 0) or (df < 0 and random.random() < math.pow(math.e, df / T)):
        best_state = candidate
        best_state_fitness = candidate_fitness
        best_states.append(( best_state_fitness, best_state ))
        if len(best_states) > bestRetention: best_states.pop(0)
  
  for state in best_states: print(state)
  print(f"\nFinal Fitness: {best_state_fitness}")
  return best_state, best_states

In [5]:
quadgram_data = { }
quadgram_data_total = 0

with open("data/quadgrams.txt") as f:
  lines = f.read().splitlines()
  for line in lines:
    s = line.split(" ")
    quadgram_data[s[0]] = int(s[1])
    quadgram_data_total += quadgram_data[s[0]]

def english_check(word):
  quadgrams = []
  for i in range(len(word) - 3):
    quadgrams.append(word[i:i+4])
  probs = [ math.log10(quadgram_data.get(q, 1) / quadgram_data_total) for q in quadgrams ]
  return sum(probs)

## Decode - Simple Example

In [6]:
pt_a = "THCOQUYBAIVJEGWNKXMLPRSDFZ"
ct_a = "3FRBEOG0NS7A4K1HXQL2VMZUYP69TDI85JWC"
print(f"Plaintext Alphabet ({len(pt_a)}): {pt_a}")
print(f"Plaintext Alphabet ({len(ct_a)}): {ct_a}")

print("")
pt_1 = "ABADHULLMAGICISAROUND"
pt_2 = "BBDHULLMAGIKISAROUND"
print(f"Plaintext 1: { pt_1 }")
print(f"Plaintext 2: { pt_2 }")

print("")
ct_1 = "".join(encode_wadsworth(pt_a, ct_a, pt_1))
ct_2 = "".join(encode_wadsworth(pt_a, ct_a, pt_2))
print(f"Ciphertext 1: { ct_1 }")
print(f"Ciphertext 2: { ct_2 }")

print("")
d_pt_1 = "".join(encode_wadsworth(ct_a, pt_a, ct_1))
d_pt_2 = "".join(encode_wadsworth(ct_a, pt_a, ct_2))
print(f"Decoded Plaintext 1: { d_pt_1 }")
print(f"Decoded Plaintext 2: { d_pt_2 }")

Plaintext Alphabet (26): THCOQUYBAIVJEGWNKXMLPRSDFZ
Plaintext Alphabet (36): 3FRBEOG0NS7A4K1HXQL2VMZUYP69TDI85JWC

Plaintext 1: ABADHULLMAGICISAROUND
Plaintext 2: BBDHULLMAGIKISAROUND

Ciphertext 1: NJWKQMCP1ICMEAY3KMUJO
Ciphertext 2: 0JKQMCP1ICMTAY3KMUJO

Decoded Plaintext 1: ABADHULLMAGICISAROUND
Decoded Plaintext 2: BBDHULLMAGIKISAROUND


## Decode - Discord Challenge

In [8]:
cts = [
  [61, 51, 71, 52, 47, 35, 59, 26, 62, 1, 11, 71, 58, 60, 45, 15, 78, 33, 46, 14, 55, 57, 7, 32, 51, 65, 71, 4, 39, 50, 0, 32, 74, 5, 3, 35, 7, 45, 29, 74, 64, 47, 76, 31, 36, 4, 72, 61, 12, 13, 81, 80, 14, 79, 5, 2, 12, 28, 8, 52, 20, 73, 32, 41, 78, 42, 50, 12, 62, 37, 40, 55, 57, 45, 56, 64, 70, 73, 44, 71, 42, 22, 24, 63, 17, 37, 27, 47, 76, 44, 77, 11, 79, 3, 30, 10, 29, 48, 27, 25, 57, 61, 78, 23, 14, 27, 66, 30, 0, 28, 81, 11, 65, 27, 58, 19, 76, 45, 31, 82, 39, 58, 35, 13, 44, 34, 23, 42, 39, 35, 73, 43, 23, 67, 6, 73, 51, 71, 52, 5, 20, 32, 74, 38, 52, 39, 19, 68, 9, 53, 21, 46, 54, 24, 61, 0, 13, 53, 80, 25, 20, 2, 51, 8, 65, 22, 20, 7, 44, 11, 33, 79, 39, 55, 68, 63, 44, 62, 31, 65, 54, 50, 63, 7, 56, 54, 35, 10, 36, 74, 14, 35, 69, 44, 62, 36, 67, 47, 60, 7, 28, 11, 79, 4, 16, 12, 81, 11, 38, 27, 50, 30, 15, 80, 79, 55, 30, 17, 44, 41, 36, 37, 46, 47, 16, 75, 7, 34, 33, 79, 72, 19, 75, 17, 77, 64, 42, 19, 12, 56, 54, 58, 73, 10, 56, 42, 3, 17, 53, 42, 5, 30, 29, 67, 82, 50, 18, 51, 48, 40, 82, 57, 61, 77, 51, 23, 67, 25, 16, 35, 69, 34, 74, 79, 72, 22, 76, 45, 34, 79, 6, 60, 61, 10, 43, 34, 71, 39, 6, 19, 30, 45, 78, 71, 46, 66, 45, 8, 27, 39, 66, 59, 13, 37, 39, 3, 68, 9, 15, 56, 52, 70, 76, 61, 41, 53, 80, 64, 58, 16, 17, 36, 1, 40, 38, 75, 69],
  [61, 51, 71, 52, 47, 35, 59, 26, 62, 1, 11, 71, 52, 72, 69, 28, 21, 54, 35, 73, 12, 13, 44, 31, 80, 14, 66, 60, 28, 43, 1, 11, 79, 38, 19, 61, 7, 29, 80, 46, 16, 57, 63, 17, 81, 11, 4, 39, 16, 30, 12, 37, 38, 35, 75, 41, 31, 54, 52, 72, 16, 2, 69, 32, 28, 29, 21, 64, 19, 75, 17, 49, 67, 72, 66, 9, 12, 77, 80, 6, 73, 32, 29, 21, 14, 3, 59, 15, 8, 4, 66, 7, 77, 33, 38, 20, 17, 49, 1, 39, 3, 57, 2, 0, 74, 55, 73, 0, 21, 33, 23, 82, 25, 18, 30, 76, 53, 48, 8, 39, 6, 2, 41, 56, 71, 82, 68, 35, 26, 17, 45, 51, 67, 70, 68, 45, 56, 23, 52, 47, 66, 76, 41, 56, 21, 65, 4, 6, 60, 9, 15, 74, 40, 72, 26, 41, 53, 80, 64, 58, 16, 17, 36, 1, 40, 38, 54, 16, 2, 44, 29, 37, 33, 14, 22, 61, 31, 81, 71, 52, 70, 61, 0, 37, 21, 42, 55, 35, 2, 7, 77, 36, 54, 4, 75, 59, 29, 74, 79, 4, 60, 77, 56, 74, 79, 20, 7, 43, 77, 81, 65, 5, 47, 0, 8, 40, 58, 22, 7, 12, 78, 81, 48, 40, 52, 55, 50, 63, 43, 74, 64, 52, 50, 28, 29, 8, 80, 22, 57, 73, 63, 28, 31, 14, 58, 70, 7, 15, 78, 38, 58, 2, 69, 34, 53, 21, 64, 67, 52, 58, 30, 61, 77, 36, 82, 27, 20, 10, 29, 11, 40, 66, 7, 17, 41, 80, 67, 19, 16, 59, 45, 34, 8, 80, 72, 75],
  [61, 51, 71, 52, 47, 35, 59, 41, 65, 71, 70, 57, 26, 44, 11, 52, 60, 9, 76, 43, 62, 56, 48, 64, 67, 25, 35, 60, 10, 36, 74, 5, 66, 73, 61, 7, 44, 81, 11, 25, 47, 63, 44, 56, 54, 4, 5, 35, 59, 9, 17, 28, 13, 81, 4, 72, 18, 7, 17, 15, 34, 78, 48, 65, 79, 55, 5, 72, 73, 69, 62, 1, 65, 42, 75, 10, 44, 21, 1, 42, 54, 6, 57, 45, 77, 21, 58, 25, 59, 32, 28, 10, 37, 42, 47, 76, 44, 74, 38, 20, 63, 29, 80, 5, 59, 77, 37, 80, 79, 42, 70, 50, 75, 17, 77, 51, 1, 25, 26, 49, 1, 6, 22, 19, 2, 69, 29, 34, 56, 38, 54, 72, 66, 50, 30, 41, 53, 82, 39, 73, 15, 67, 46, 4, 19, 57, 77, 65, 4, 68, 20, 43, 53, 21, 42, 68, 9, 53, 79, 25, 20, 57, 61, 41, 31, 38, 39, 18, 17, 28, 31, 82, 68, 57, 18, 12, 51, 1, 46, 50, 32, 49, 11, 71, 68, 47, 7, 36, 37, 39, 59, 9, 45, 37, 42, 35, 43, 29, 1, 72, 70, 35, 76, 62, 64, 22, 3, 57, 26, 9, 32, 34, 37, 27, 70, 73, 32, 53, 14, 79, 4, 47, 24, 18, 69, 12, 51, 36, 46, 16, 9, 41, 36, 48, 40, 16, 60, 28, 48, 67, 6, 16, 69, 41, 80, 14, 3, 50, 7, 62, 31, 42, 19, 12, 49, 53, 67, 5, 16, 69, 34, 46, 39, 22, 3, 57, 73, 15, 37],
  [61, 51, 71, 52, 47, 35, 59, 41, 65, 71, 22, 19, 7, 32, 31, 82, 70, 57, 7, 48, 4, 6, 60, 44, 48, 80, 54, 4, 47, 35, 45, 32, 37, 14, 68, 16, 32, 78, 4, 47, 32, 10, 53, 81, 74, 1, 58, 22, 18, 15, 8, 40, 52, 18, 0, 49, 64, 4, 75, 57, 73, 0, 77, 64, 3, 47, 76, 61, 77, 62, 56, 67, 6, 66, 76, 78, 81, 38, 70, 22, 3, 69, 7, 12, 49, 14, 27, 73, 32, 29, 80, 5, 73, 15, 37, 82, 68, 61, 77, 33, 27, 75, 28, 81, 4, 6, 24, 69, 62, 65, 82, 25, 6, 60, 51, 71, 3, 60, 15, 49, 53, 21, 40, 22, 19, 20, 0, 51, 21, 80, 72, 47, 66, 76, 43, 56, 80, 64, 38, 68, 18, 61, 56, 54, 55, 25, 69, 31, 33, 54, 55, 59, 0, 15, 74, 54, 16, 0, 8, 40, 25, 47, 0, 44, 78, 1, 64, 55, 2, 60, 77, 49, 56, 42, 70, 47, 7, 36, 79, 58, 60, 43, 36, 80, 79, 35, 57, 76, 17, 29, 46, 70, 50, 57, 69, 13, 37, 71, 68, 12, 51, 8, 33, 70, 22, 63, 12, 41, 62, 29, 37, 8, 79, 27, 5, 58, 73, 41, 29, 36, 81, 54, 16, 73, 43, 33, 6, 35, 10, 62, 33, 64, 52, 25, 63, 10, 77, 49, 8, 14, 54, 25, 63, 17, 15, 78, 21, 42, 47, 57, 12, 13, 78, 81, 67, 42, 47, 66, 32, 8, 74, 4, 68, 61, 28, 74, 67, 40, 55, 22, 12, 0, 28, 31, 21, 33, 54, 19, 60, 13, 62, 81, 46, 6, 60, 51, 81, 23, 46, 50, 59, 76, 45, 51, 8, 52, 19, 66, 10, 36, 21, 5, 19, 26, 0, 48, 37, 71, 82, 38, 6, 19, 7, 77]
]

pt_a = "BDMAGICKEFHJLNOPQRSTUVWXYZ"
ct_a = [ 63, 69, 76, 61, 7, 12, 17, 45, 0, 32, 28, 10, 43, 13, 44, 41, 77, 51, 15, 62, 29, 49, 34, 53, 31, 36, 78, 81, 56, 48, 37, 8, 74, 21, 1, 11, 33, 80, 23, 65, 71, 64, 67, 40, 46, 14, 79, 42, 82, 38, 54, 4, 52, 27, 39, 55, 5, 58, 25, 6, 72, 70, 22, 3, 19, 68, 47, 16, 66, 50, 20, 35, 75, 57, 2, 24, 60, 18, 73, 30, 59, 26, 9 ]

ct_1 = cts[0]
d_pt_1 = encode_wadsworth(ct_a, pt_a, ct_1)
print("".join(d_pt_1))

ct_2 = [28,27,71,18,80,27,26,76,47,48,21,38,15,60,33,20,43,40,66,75,29,24,11,4,60,72,19,41,36,35,55,6,79,33,25,63,38,6,77,27,26,2,38,21,80,23,47,16,2,77,6,45,74,33,16,65,9,41,30,72,74,58,51,33,20,4,11,28,27,7,46,26,27,34,61,40,49,57,82,38,35,1,2,77,11,36,43,52,81,19,35,79,59,38,41,70,17,1,31,71,61,40,37,16,17,8,18,56,68,75,64,47,30,65,52,2,55,35,71,37,38,6,62,43,77,64,45,11,58,78,82,51,15,8,63,25,67,48,47,30,65,53,28,22,75,30,81,8,63,57,49,55,2,16,15,1,54,26,64,34,6,75,57,21,25,14,20,65,20,4,55,61,56,74,3,11,73,10,81,57,31,9,67,55,14,48,28,59,71,22,62,29,72,51,21,52,49,10,67,60,43,12,65,55,49,76,50,10,39,16,21,19,82,53,4,30,50,51,67,79,18,28,27,80,64,58,29,24,11,47,78,65,19,21,19,58,60,14,11,34,17,56,32,25,81,76,4,52,49,10,65,78,77,63,62,63,36,43,38,32,36,15,25,41,62,63,13,22,45,29,72,64,75,74,24,9,50,76,82,23,73,42,73,29,31,55,81,20,61,62,68,77,2,1,42,80,49,10,73,9,67,30,14,44,7,74,65,0,47,44,7,74,34,78,3,12,82,70,81,25,67,51,14,57,32,30,21]
d_pt_2 = encode_wadsworth(ct_a, pt_a, ct_2, lambda d,_1,_2: ((d - 1) * 28) % 83)
print("".join(d_pt_2))


AROBOTMAYNOTHARMHUMANITYCMAORCMABYINACTIONCMAALLOWHUMANITYTOCOMETOHARMPDTHISLAWWASINTRODUCEDLATERBYASIMOVASAPREQUELTOTHEORIGINALTHREELAWSPDITPLACESTHESAFETYANDWELLBEINGOFHUMANITYASAWHOLEABOVEEVENTHESAFETYOFINDIVIDUALHUMANSCMAEXPANDINGONTHEIDEATHATTHEROLEOFROBOTSISTOSERVEANDPROTECTHUMANITYASAWHOLECMARATHERTHANJUSTINDIVIDUALSPD
AROBOTMAYNOTHARMHUMANITYCMAORCMABYINACTIONCMAALLOWHUMANITYTOCOMETOHARMPDTHISLAWWASINTRODUCEDLATERBYASIMOVASAPREQUELTOTHEORIGINALTHREELAWSPDITPLACESTHESAFETYANDWELLBEINGOFHUMANITYASAWHOLEABOVEEVENTHESAFETYOFINDIVIDUALHUMANSCMAEXPANDINGONTHEIDEATHATTHEROLEOFROBOTSISTOSERVEANDPROTECTHUMANITYASAWHOLECMARATHERTHANJUSTINDIVIDUALSPD


## Simulated Annealing - Numbers

In [9]:
f_num_init = lambda : random.uniform(-100, 100)
f_num_nudge = lambda x : x + random.uniform(-5, 5)
f_num_fitness_fc = lambda t : lambda x : 1.0 / abs(t - x)

best, best_list = simulate_annealing(10, 100, 200, 10, f_num_init, f_num_nudge, f_num_fitness_fc(-50))
print(best)

(0.13955170788782262, -57.165802662937246)
(0.10847508960557384, -59.21870637430307)
(0.21905208840554144, -54.56512424637857)
(0.42858417022111905, -52.33326396419184)
(0.6821585286339137, -51.465934908125526)
(162.3862322728882, -49.993841842464086)
(173.74904046597646, -50.00575542746779)
(3135.441080909064, -50.00031893439366)
(3138.382690800752, -49.99968136454393)
(3635.5418219240896, -49.999724937836234)

Final Fitness: 3635.5418219240896
-49.999724937836234


## Simulated Annealing - Wadsworth

In [10]:
def ffac_wadsworth_init(
    pt_a = list(string.ascii_uppercase),
    ct_a = list(string.ascii_uppercase + "0123456789")
):
  return lambda : ( pt_a, ct_a )

def f_wadsworth_nudge(a):
  pt_a = a[0].copy()
  ct_a = a[1].copy()

  if random.random() < 0.5:
    i1, i2 = random.sample(range(len(pt_a)), 2)
    pt_a[i1], pt_a[i2] = pt_a[i2], pt_a[i1]
  else:
    i1, i2 = random.sample(range(len(ct_a)), 2)
    ct_a[i1], ct_a[i2] = ct_a[i2], ct_a[i1]

  return ( pt_a, ct_a )

def ffac_wadsworth_fitness(
    ct = "NJWKQMCP1ICMEAY3KMUJO",
    toLog=False
):
  def f_wadsworth_fitness(a):
    d_pt = encode_wadsworth(a[1], a[0], ct)
    # fitness = 1.0 / np.std([ (ord(d_pt[i]) - ord(d_pt[i + 1])) % len(a[1]) for i in range(len(d_pt) - 1) ])
    fitness = english_check("".join(d_pt))
    if toLog: print(f"{fitness} : {d_pt}")
    return fitness
  return f_wadsworth_fitness


In [11]:
print("Expected Outcome")
print("----------------")
pt_a = "THCOQUYBAIVJEGWNKXMLPRSDFZ"
ct_a = "3FRBEOG0NS7A4K1HXQL2VMZUYP69TDI85JWC"
ct = "NJWKQMCP1ICMEAY3KMUJO"
fitness = ffac_wadsworth_fitness(ct, toLog=True)(( pt_a, ct_a ))

print("\nSimulated Annealing")
print("-------------------")
pt_a = list(string.ascii_uppercase)
ct_a = list(string.ascii_uppercase + "0123456789")
ct = "NJWKQMCP1ICMEAY3KMUJO"
f_init = ffac_wadsworth_init(pt_a, ct_a)
f_fitness = ffac_wadsworth_fitness(ct)

final_a, best_a = simulate_annealing(10, 100, 2000, 10, f_init, f_wadsworth_nudge, f_fitness)
fitness = ffac_wadsworth_fitness(ct, True)(final_a)


Expected Outcome
----------------
-86.8152826643564 : ['A', 'B', 'A', 'D', 'H', 'U', 'L', 'L', 'M', 'A', 'G', 'I', 'C', 'I', 'S', 'A', 'R', 'O', 'U', 'N', 'D']

Simulated Annealing
-------------------
(-60.72434967738624, (['I', 'T', 'H', 'N', 'W', 'D', 'Y', 'E', 'L', 'P', 'Z', 'Q', 'G', 'M', 'X', 'A', 'U', 'F', 'J', 'B', 'S', 'K', 'O', 'C', 'V', 'R'], ['Z', 'U', 'B', 'S', '0', '2', 'W', 'L', '8', 'K', 'J', 'T', '3', 'P', '1', 'G', 'D', '9', 'O', 'I', 'N', 'X', '4', '5', '7', 'R', 'V', 'E', 'A', 'C', '6', 'H', 'F', 'Q', 'M', 'Y']))
(-60.625394124842856, (['I', 'T', 'H', 'N', 'W', 'D', 'Y', 'E', 'L', 'P', 'Z', 'Q', 'G', 'M', 'X', 'A', 'U', 'F', 'J', 'B', 'S', 'K', 'O', 'C', 'V', 'R'], ['Z', 'U', 'B', 'S', '0', '2', 'W', 'L', '8', 'K', 'J', 'T', '3', 'P', '1', 'N', 'D', '9', 'O', 'I', 'G', 'X', '4', '5', '7', 'R', 'V', 'E', 'A', 'C', '6', 'H', 'F', 'Q', 'M', 'Y']))
(-60.27863200830127, (['I', 'T', 'H', 'N', 'W', 'D', 'Y', 'E', 'L', 'P', 'Z', 'Q', 'G', 'M', 'X', 'A', 'U', 'F', 'J', 'B', '

In [12]:
print("Expected Outcome")
print("----------------")
pt_a = "BDMAGICKEFHJLNOPQRSTUVWXYZ"
ct_a = [ 63, 69, 76, 61, 7, 12, 17, 45, 0, 32, 28, 10, 43, 13, 44, 41, 77, 51, 15, 62, 29, 49, 34, 53, 31, 36, 78, 81, 56, 48, 37, 8, 74, 21, 1, 11, 33, 80, 23, 65, 71, 64, 67, 40, 46, 14, 79, 42, 82, 38, 54, 4, 52, 27, 39, 55, 5, 58, 25, 6, 72, 70, 22, 3, 19, 68, 47, 16, 66, 50, 20, 35, 75, 57, 2, 24, 60, 18, 73, 30, 59, 26, 9 ]
ct_1 = [61, 51, 71, 52, 47, 35, 59, 26, 62, 1, 11, 71, 58, 60, 45, 15, 78, 33, 46, 14, 55, 57, 7, 32, 51, 65, 71, 4, 39, 50, 0, 32, 74, 5, 3, 35, 7, 45, 29, 74, 64, 47, 76, 31, 36, 4, 72, 61, 12, 13, 81, 80, 14, 79, 5, 2, 12, 28, 8, 52, 20, 73, 32, 41, 78, 42, 50, 12, 62, 37, 40, 55, 57, 45, 56, 64, 70, 73, 44, 71, 42, 22, 24, 63, 17, 37, 27, 47, 76, 44, 77, 11, 79, 3, 30, 10, 29, 48, 27, 25, 57, 61, 78, 23, 14, 27, 66, 30, 0, 28, 81, 11, 65, 27, 58, 19, 76, 45, 31, 82, 39, 58, 35, 13, 44, 34, 23, 42, 39, 35, 73, 43, 23, 67, 6, 73, 51, 71, 52, 5, 20, 32, 74, 38, 52, 39, 19, 68, 9, 53, 21, 46, 54, 24, 61, 0, 13, 53, 80, 25, 20, 2, 51, 8, 65, 22, 20, 7, 44, 11, 33, 79, 39, 55, 68, 63, 44, 62, 31, 65, 54, 50, 63, 7, 56, 54, 35, 10, 36, 74, 14, 35, 69, 44, 62, 36, 67, 47, 60, 7, 28, 11, 79, 4, 16, 12, 81, 11, 38, 27, 50, 30, 15, 80, 79, 55, 30, 17, 44, 41, 36, 37, 46, 47, 16, 75, 7, 34, 33, 79, 72, 19, 75, 17, 77, 64, 42, 19, 12, 56, 54, 58, 73, 10, 56, 42, 3, 17, 53, 42, 5, 30, 29, 67, 82, 50, 18, 51, 48, 40, 82, 57, 61, 77, 51, 23, 67, 25, 16, 35, 69, 34, 74, 79, 72, 22, 76, 45, 34, 79, 6, 60, 61, 10, 43, 34, 71, 39, 6, 19, 30, 45, 78, 71, 46, 66, 45, 8, 27, 39, 66, 59, 13, 37, 39, 3, 68, 9, 15, 56, 52, 70, 76, 61, 41, 53, 80, 64, 58, 16, 17, 36, 1, 40, 38, 75, 69]
fitness = ffac_wadsworth_fitness(ct_1, toLog=True)(( pt_a, ct_a ))

print("\nSimulated Annealing")
print("-------------------")
# pt_a = list(string.ascii_uppercase)
# ct_a = list(range(83))
pt_a = list("BDMAGICKEFHJLNOPQRSTUVWXYZ")
ct_a = [ 63, 69, 76, 61, 7, 12, 17, 45, 0, 32, 28, 10, 43, 13, 44, 41, 77, 51, 15, 62, 29, 49, 34, 53, 31, 36, 78, 81, 56, 48, 37, 8, 74, 21, 1, 11, 33, 80, 23, 65, 71, 64, 67, 40, 46, 14, 79, 42, 82, 38, 54, 4, 52, 27, 39, 55, 5, 58, 25, 6, 72, 70, 22, 3, 19, 68, 47, 16, 66, 50, 20, 35, 75, 57, 2, 24, 60, 18, 73, 30, 59, 26, 9 ]
f_init = ffac_wadsworth_init(pt_a, ct_a)
f_fitness = ffac_wadsworth_fitness(ct_1)
final_a, best_a = simulate_annealing(10, 25, 2000, 30, f_init, f_wadsworth_nudge, f_fitness)
fitness = ffac_wadsworth_fitness(ct_1, True)(final_a)

# ct_2 = [28,27,71,18,80,27,26,76,47,48,21,38,15,60,33,20,43,40,66,75,29,24,11,4,60,72,19,41,36,35,55,6,79,33,25,63,38,6,77,27,26,2,38,21,80,23,47,16,2,77,6,45,74,33,16,65,9,41,30,72,74,58,51,33,20,4,11,28,27,7,46,26,27,34,61,40,49,57,82,38,35,1,2,77,11,36,43,52,81,19,35,79,59,38,41,70,17,1,31,71,61,40,37,16,17,8,18,56,68,75,64,47,30,65,52,2,55,35,71,37,38,6,62,43,77,64,45,11,58,78,82,51,15,8,63,25,67,48,47,30,65,53,28,22,75,30,81,8,63,57,49,55,2,16,15,1,54,26,64,34,6,75,57,21,25,14,20,65,20,4,55,61,56,74,3,11,73,10,81,57,31,9,67,55,14,48,28,59,71,22,62,29,72,51,21,52,49,10,67,60,43,12,65,55,49,76,50,10,39,16,21,19,82,53,4,30,50,51,67,79,18,28,27,80,64,58,29,24,11,47,78,65,19,21,19,58,60,14,11,34,17,56,32,25,81,76,4,52,49,10,65,78,77,63,62,63,36,43,38,32,36,15,25,41,62,63,13,22,45,29,72,64,75,74,24,9,50,76,82,23,73,42,73,29,31,55,81,20,61,62,68,77,2,1,42,80,49,10,73,9,67,30,14,44,7,74,65,0,47,44,7,74,34,78,3,12,82,70,81,25,67,51,14,57,32,30,21]
# d_pt_2 = encode_wadsworth(ct_a, pt_a, ct_2, lambda d,_1,_2: ((d - 1) * 28) % 83)
# print("".join(d_pt_2))


Expected Outcome
----------------
-1444.2570309244127 : ['A', 'R', 'O', 'B', 'O', 'T', 'M', 'A', 'Y', 'N', 'O', 'T', 'H', 'A', 'R', 'M', 'H', 'U', 'M', 'A', 'N', 'I', 'T', 'Y', 'C', 'M', 'A', 'O', 'R', 'C', 'M', 'A', 'B', 'Y', 'I', 'N', 'A', 'C', 'T', 'I', 'O', 'N', 'C', 'M', 'A', 'A', 'L', 'L', 'O', 'W', 'H', 'U', 'M', 'A', 'N', 'I', 'T', 'Y', 'T', 'O', 'C', 'O', 'M', 'E', 'T', 'O', 'H', 'A', 'R', 'M', 'P', 'D', 'T', 'H', 'I', 'S', 'L', 'A', 'W', 'W', 'A', 'S', 'I', 'N', 'T', 'R', 'O', 'D', 'U', 'C', 'E', 'D', 'L', 'A', 'T', 'E', 'R', 'B', 'Y', 'A', 'S', 'I', 'M', 'O', 'V', 'A', 'S', 'A', 'P', 'R', 'E', 'Q', 'U', 'E', 'L', 'T', 'O', 'T', 'H', 'E', 'O', 'R', 'I', 'G', 'I', 'N', 'A', 'L', 'T', 'H', 'R', 'E', 'E', 'L', 'A', 'W', 'S', 'P', 'D', 'I', 'T', 'P', 'L', 'A', 'C', 'E', 'S', 'T', 'H', 'E', 'S', 'A', 'F', 'E', 'T', 'Y', 'A', 'N', 'D', 'W', 'E', 'L', 'L', 'B', 'E', 'I', 'N', 'G', 'O', 'F', 'H', 'U', 'M', 'A', 'N', 'I', 'T', 'Y', 'A', 'S', 'A', 'W', 'H', 'O', 'L', 'E', 'A', 'B', 'O'

## Simulated Annealing - Eyes

In [13]:
messages = [
    [50, 66, 5, 48, 62, 13, 75, 29, 24, 61, 42, 70, 66, 62, 32, 14, 81, 8, 15, 78, 2, 29, 13, 49, 1, 80, 82, 40, 63, 81, 21, 19, 0, 40, 51, 65, 26, 14, 21, 70, 47, 44, 48, 42, 19, 48, 13, 47, 19, 49, 72, 31, 5, 24, 3, 43, 59, 67, 33, 49, 41, 60, 21, 26, 30, 5, 25, 20, 71, 11, 74, 56, 4, 74, 19, 71, 4, 51, 41, 43, 80, 72, 54, 63, 79, 81, 15, 16, 44, 31, 30, 12, 33, 57, 28, 13, 64, 43, 48],
    [80, 66, 5, 48, 62, 13, 75, 29, 24, 61, 42, 70, 66, 62, 32, 14, 81, 8, 15, 78, 2, 29, 13, 49, 1, 29, 11, 30, 52, 81, 21, 19, 0, 25, 26, 54, 20, 14, 21, 70, 47, 44, 48, 42, 19, 48, 13, 47, 19, 49, 44, 26, 59, 77, 64, 43, 79, 28, 72, 64, 1, 30, 73, 23, 67, 6, 33, 25, 64, 81, 68, 46, 17, 36, 13, 17, 21, 68, 13, 9, 46, 67, 57, 34, 62, 82, 15, 10, 73, 62, 2, 11, 65, 72, 37, 44, 10, 43, 68, 62, 9, 34, 18],
    [36, 66, 5, 48, 62, 13, 75, 29, 24, 61, 42, 70, 66, 62, 32, 14, 81, 8, 15, 78, 2, 29, 13, 49, 1, 69, 76, 52, 9, 48, 66, 80, 22, 64, 57, 40, 49, 78, 3, 16, 56, 19, 47, 40, 80, 6, 13, 64, 29, 49, 64, 63, 6, 49, 31, 13, 16, 10, 45, 24, 26, 77, 10, 60, 81, 61, 34, 54, 70, 21, 15, 4, 66, 77, 42, 37, 30, 22, 0, 11, 41, 72, 57, 20, 23, 57, 65, 41, 23, 18, 72, 42, 5, 3, 26, 78, 8, 5, 54, 45, 77, 25, 64, 61, 16, 44, 54, 51, 20, 63, 25, 11, 26, 45, 53, 60, 38, 34],
    [76, 66, 5, 49, 75, 54, 69, 46, 32, 1, 42, 60, 26, 48, 50, 80, 32, 24, 55, 61, 47, 12, 21, 12, 49, 54, 34, 25, 36, 15, 56, 55, 20, 9, 8, 62, 13, 82, 9, 44, 29, 60, 53, 82, 42, 80, 5, 43, 71, 3, 80, 77, 47, 78, 34, 25, 62, 18, 10, 49, 62, 64, 52, 81, 11, 66, 62, 13, 47, 17, 52, 70, 26, 23, 32, 31, 64, 23, 35, 32, 50, 6, 1, 25, 8, 37, 47, 43, 26, 76, 65, 68, 80, 17, 7, 45, 63, 14, 53, 63, 60, 16],
    [63, 66, 5, 49, 75, 54, 2, 60, 29, 40, 78, 47, 60, 75, 67, 71, 60, 2, 65, 7, 47, 14, 45, 74, 59, 41, 80, 13, 60, 13, 81, 22, 35, 50, 40, 39, 2, 59, 48, 31, 76, 2, 80, 75, 1, 56, 67, 11, 21, 8, 40, 65, 45, 75, 55, 39, 60, 42, 13, 3, 22, 57, 2, 6, 58, 9, 70, 1, 58, 56, 63, 68, 25, 79, 7, 20, 19, 64, 2, 66, 73, 30, 71, 16, 12, 30, 65, 37, 20, 13, 22, 63, 18, 46, 64, 59, 41, 81, 82, 22, 78, 36, 47, 17, 4, 6, 17, 5, 36, 79, 63, 1, 64, 69, 15, 43, 4, 58, 56, 31, 14, 64, 58, 18, 44, 78, 69, 1, 0, 46, 20, 71, 73, 25, 35, 8, 24],
    [34, 66, 5, 49, 75, 54, 23, 74, 11, 13, 28, 26, 19, 48, 67, 57, 37, 60, 34, 28, 74, 10, 17, 32, 11, 18, 19, 43, 19, 81, 42, 4, 62, 9, 46, 49, 32, 51, 76, 58, 4, 43, 47, 17, 67, 79, 21, 32, 44, 16, 30, 37, 26, 28, 41, 68, 57, 34, 51, 10, 69, 70, 8, 6, 46, 43, 18, 39, 47, 43, 15, 13, 33, 30, 35, 62, 37, 0, 37, 5, 38, 55, 37, 13, 40, 25, 9, 21, 11, 64, 5, 79, 42, 68, 11, 71, 11, 48, 3, 67, 61, 40, 22, 14, 35, 50, 61, 39, 11, 2, 66, 49, 51, 53, 17, 73, 36, 75, 74, 54, 24, 30, 54, 70],
    [27, 66, 5, 49, 75, 54, 2, 60, 29, 40, 2, 55, 9, 15, 59, 18, 68, 3, 36, 5, 47, 77, 44, 38, 1, 18, 28, 76, 4, 34, 60, 63, 58, 80, 17, 54, 79, 75, 48, 54, 55, 19, 62, 64, 14, 47, 51, 70, 75, 5, 11, 47, 45, 58, 68, 69, 79, 25, 38, 45, 73, 47, 68, 50, 34, 45, 78, 26, 79, 57, 4, 56, 22, 60, 18, 75, 43, 60, 59, 67, 63, 42, 49, 33, 40, 65, 79, 77, 7, 3, 26, 62, 31, 78, 26, 57, 69, 40, 4, 23, 26, 13, 67, 42, 38, 72, 11, 39, 65, 60, 25, 6, 80, 66, 68, 77, 59, 78, 19],
    [77, 66, 5, 49, 75, 54, 2, 60, 29, 40, 2, 55, 9, 15, 59, 18, 68, 3, 36, 5, 47, 60, 21, 80, 1, 72, 55, 16, 82, 35, 57, 19, 1, 66, 18, 27, 39, 17, 74, 81, 39, 14, 78, 0, 25, 65, 43, 66, 64, 38, 81, 23, 24, 50, 57, 30, 71, 75, 26, 68, 54, 57, 56, 50, 71, 73, 14, 21, 8, 32, 26, 63, 5, 37, 19, 43, 66, 47, 53, 34, 66, 23, 73, 31, 54, 38, 77, 67, 11, 63, 79, 6, 22, 21, 51, 69, 74, 21, 5, 17, 67, 37, 29, 21, 60, 14, 82, 44, 30, 4, 20, 42, 35, 1, 31, 54, 46, 20, 40, 30],
    [33, 66, 5, 49, 75, 54, 2, 60, 29, 40, 2, 55, 9, 15, 59, 18, 68, 3, 36, 5, 47, 33, 21, 59, 44, 18, 28, 76, 59, 34, 60, 63, 79, 27, 12, 54, 5, 49, 48, 54, 55, 52, 62, 72, 69, 10, 57, 22, 58, 48, 67, 53, 7, 34, 32, 30, 31, 19, 26, 8, 34, 46, 7, 30, 71, 55, 34, 75, 54, 9, 6, 60, 5, 23, 25, 45, 42, 80, 25, 12, 22, 76, 20, 51, 62, 21, 40, 9, 41, 10, 44, 73, 8, 33, 70, 73, 6, 31, 21, 72, 5, 40, 61, 51, 42, 66, 64, 74, 61, 25, 63, 42, 24, 41]
]

ct_m1 = messages[0]
d_pt_m1 = encode_wadsworth(ct_a, pt_a, ct_m1, lambda d,ct,_2 : d * ct[0])
print("".join(d_pt_m1))


SHYQBMOGYMSQMLWMMUHUMWBHHGEECGSECOGLCQGBQCMSHSOLCGGOMQGSLBMWYCGLCQLOLLECLOMOSYSOQWCSGCOULSLUHOBUQLG
